In [97]:
import csv
import pandas as pd
import os
from sys import argv
from matplotlib import pyplot as plt
import numpy as np
# from .utils import reduce_expr_matrix

In [98]:
def read_file(pkn_nodes_file:str) ->list :
    with open(pkn_nodes_file, 'r') as f:
        return f.read().splitlines()

def load_data(filename):
    return pd.read_csv(filename, compression='zip')

In [99]:
pkn_nodes_file = '../pkn_data/nodes.txt'
pkn_no_predecessors_file = '../pkn_data/no_predecessors.txt'
pkn_no_successors_file = '../pkn_data/no_successors.txt'
genes_list = read_file(pkn_nodes_file)
inputs_list = read_file(pkn_no_predecessors_file)
readouts_list = read_file(pkn_no_successors_file)

raw_mtx_file = '../../raw_mtx_merged.zip'
raw_mtx = load_data(raw_mtx_file)
raw_mtx = raw_mtx.set_index('Name')
raw_mtx.to_csv('../matrices/real_datasets/raw_mtx.csv', index=True)



In [100]:
annotations = raw_mtx.iloc[:,:7]
annotations

,Embryo,clusterUmap,Pseudotime,Dataset,EmbryoDay,Stage,clusterEmbryoCells
Name,,,,,,,
E4.1.1,E4.1,Morula,15.261600,Petropoulos2016,4.0,M,K03
E4.1.2,E4.1,Morula,17.434218,Petropoulos2016,4.0,M,K03
E4.1.3,E4.1,Morula,16.549153,Petropoulos2016,4.0,M,K03
E4.1.4,E4.1,Morula,13.866676,Petropoulos2016,4.0,M,K03
E4.1.7,E4.1,Morula,20.693400,Petropoulos2016,4.0,M,K03
...,...,...,...,...,...,...,...
E7.9.568,E7.9,late_TE,52.046244,Petropoulos2016,7.0,B,K08
E7.9.569,E7.9,EPI,40.336750,Petropoulos2016,7.0,B,K08
E7.9.570,E7.9,medium_TE,45.582031,Petropoulos2016,7.0,B,K08


In [101]:
raw_without_annotations = raw_mtx.iloc[:,7:]
raw_without_annotations

,5S_rRNA,5_8S_rRNA,7SK,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,...,ZYG11B,ZYX,ZZEF1,ZZZ3,pRNA,snoU109,snoU13,snoU2-30,snoU2_19,uc_338
Name,,,,,,,,,,,,,,,,,,,,,
E4.1.1,0,21,0,0,0,7,0,0,0,0,...,141,447,101,184,0,0,0,0,0,5
E4.1.2,0,1,0,0,0,0,0,0,0,0,...,1,98,23,343,0,0,0,0,0,4
E4.1.3,0,0,0,0,3,0,1,0,0,0,...,0,209,0,73,0,0,13,0,0,0
E4.1.4,2,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,10
E4.1.7,0,0,0,0,0,0,0,0,4,1,...,29,35,62,135,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E7.9.568,0,5,0,0,17,0,0,0,71,0,...,4,202,17,550,0,0,0,0,0,0
E7.9.569,0,1,0,0,0,0,121,0,2,2,...,140,89,182,165,0,0,5,0,0,0
E7.9.570,0,0,0,0,0,0,0,0,0,0,...,0,59,19,401,0,0,0,0,0,0


In [102]:
column_name = raw_without_annotations.columns
index = raw_without_annotations.index
bin_mtx_without_annotations = np.where(raw_without_annotations<2, 0, 1)
bin_mtx_without_annotations = pd.DataFrame(bin_mtx_without_annotations, columns=column_name)
bin_mtx_without_annotations.index = index

bin_mtx_without_annotations


,5S_rRNA,5_8S_rRNA,7SK,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,...,ZYG11B,ZYX,ZZEF1,ZZZ3,pRNA,snoU109,snoU13,snoU2-30,snoU2_19,uc_338
Name,,,,,,,,,,,,,,,,,,,,,
E4.1.1,0,1,0,0,0,1,0,0,0,0,...,1,1,1,1,0,0,0,0,0,1
E4.1.2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,1
E4.1.3,0,0,0,0,1,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
E4.1.4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
E4.1.7,0,0,0,0,0,0,0,0,1,0,...,1,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E7.9.568,0,1,0,0,1,0,0,0,1,0,...,1,1,1,1,0,0,0,0,0,0
E7.9.569,0,0,0,0,0,0,1,0,1,1,...,1,1,1,1,0,0,1,0,0,0
E7.9.570,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0


In [103]:
bin_mtx_with_annotations = pd.concat([annotations, bin_mtx_without_annotations], axis=1)

bin_mtx_with_annotations.to_csv('../matrices/real_datasets/bin_mtx_with_annotations.csv', index=True)
bin_mtx_with_annotations

,Embryo,clusterUmap,Pseudotime,Dataset,EmbryoDay,Stage,clusterEmbryoCells,5S_rRNA,5_8S_rRNA,7SK,...,ZYG11B,ZYX,ZZEF1,ZZZ3,pRNA,snoU109,snoU13,snoU2-30,snoU2_19,uc_338
Name,,,,,,,,,,,,,,,,,,,,,
E4.1.1,E4.1,Morula,15.261600,Petropoulos2016,4.0,M,K03,0,1,0,...,1,1,1,1,0,0,0,0,0,1
E4.1.2,E4.1,Morula,17.434218,Petropoulos2016,4.0,M,K03,0,0,0,...,0,1,1,1,0,0,0,0,0,1
E4.1.3,E4.1,Morula,16.549153,Petropoulos2016,4.0,M,K03,0,0,0,...,0,1,0,1,0,0,1,0,0,0
E4.1.4,E4.1,Morula,13.866676,Petropoulos2016,4.0,M,K03,1,0,0,...,0,0,0,0,0,0,0,0,0,1
E4.1.7,E4.1,Morula,20.693400,Petropoulos2016,4.0,M,K03,0,0,0,...,1,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E7.9.568,E7.9,late_TE,52.046244,Petropoulos2016,7.0,B,K08,0,1,0,...,1,1,1,1,0,0,0,0,0,0
E7.9.569,E7.9,EPI,40.336750,Petropoulos2016,7.0,B,K08,0,0,0,...,1,1,1,1,0,0,1,0,0,0
E7.9.570,E7.9,medium_TE,45.582031,Petropoulos2016,7.0,B,K08,0,0,0,...,0,1,1,1,0,0,0,0,0,0


In [104]:

types_of_cells_selected = ['early_TE','medium_TE', 'late_TE']
TE_bin_mtx_with_annotations = bin_mtx_with_annotations[bin_mtx_with_annotations['clusterUmap'].isin(types_of_cells_selected)]
TE_annotations = TE_bin_mtx_with_annotations.iloc[:,:7]

TE_bin_mtx_with_annotations.to_csv('../matrices/real_datasets/TE_bin_mtx_with_annotations.csv', index=True)
TE_bin_mtx_with_annotations



,Embryo,clusterUmap,Pseudotime,Dataset,EmbryoDay,Stage,clusterEmbryoCells,5S_rRNA,5_8S_rRNA,7SK,...,ZYG11B,ZYX,ZZEF1,ZZZ3,pRNA,snoU109,snoU13,snoU2-30,snoU2_19,uc_338
Name,,,,,,,,,,,,,,,,,,,,,
E5.1.28,E5.1,early_TE,37.294838,Petropoulos2016,5.0,B,K07,0,0,0,...,1,1,0,1,0,0,0,0,0,0
E5.1.30,E5.1,early_TE,39.537387,Petropoulos2016,5.0,B,K07,0,0,0,...,1,1,1,1,0,0,0,0,0,0
E5.1.32,E5.1,early_TE,39.483993,Petropoulos2016,5.0,B,K07,0,0,0,...,1,1,1,0,0,0,0,0,0,0
E5.10.933,E5.10,early_TE,31.499036,Petropoulos2016,5.0,B,K06,0,0,0,...,0,1,1,1,0,0,0,0,0,0
E5.10.934,E5.10,early_TE,39.549130,Petropoulos2016,5.0,B,K06,0,0,0,...,1,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E7.9.565,E7.9,medium_TE,48.315030,Petropoulos2016,7.0,B,K08,0,1,0,...,1,1,1,1,1,0,0,0,1,0
E7.9.567,E7.9,medium_TE,45.798088,Petropoulos2016,7.0,B,K08,0,0,0,...,0,1,1,1,0,0,0,0,0,0
E7.9.568,E7.9,late_TE,52.046244,Petropoulos2016,7.0,B,K08,0,1,0,...,1,1,1,1,0,0,0,0,0,0


In [105]:

def reduce_expr_matrix(expr_mtx, genes_list):
    expr_mtx_genes = expr_mtx.columns
    almost_in_expr_mtx = list()
    not_in_expr_mtx = list()
    for gene in genes_list:
        present=False
        almost=False
        for g in expr_mtx_genes:
            if gene == g: 
                present=True
                break
            elif g.lower() in gene.lower(): 
                almost=True
                
        if not present and not almost: not_in_expr_mtx.append(gene)
        elif not present and almost : almost_in_expr_mtx.append(gene)


    for gene in not_in_expr_mtx:
        genes_list.remove(gene)
    for gene in almost_in_expr_mtx:
        genes_list.remove(gene)

    to_keep = genes_list
    reduced_expr_mtx = expr_mtx[to_keep]
    return (reduced_expr_mtx, not_in_expr_mtx, almost_in_expr_mtx)

In [106]:
PKN_genes = genes_list # Cf. first box
PKN_TE_bin_mtx,_,_ = reduce_expr_matrix(TE_bin_mtx_with_annotations, PKN_genes)
PKN_TE_bin_mtx_with_annotations =  pd.concat([TE_annotations, PKN_TE_bin_mtx], axis=1)
PKN_TE_bin_mtx_with_annotations.to_csv('../matrices/real_datasets/PKN_TE_bin_mtx_with_annotations.csv', index=True)
PKN_TE_bin_mtx_with_annotations

,Embryo,clusterUmap,Pseudotime,Dataset,EmbryoDay,Stage,clusterEmbryoCells,CDX2,LGALS1,CD3D,...,CNR2,E2F4,CGB3,YY1,EDN1,JAK3,STAT3,CGB8,RBPJ,SOX30
Name,,,,,,,,,,,,,,,,,,,,,
E5.1.28,E5.1,early_TE,37.294838,Petropoulos2016,5.0,B,K07,1,1,0,...,0,1,0,1,0,0,0,0,1,0
E5.1.30,E5.1,early_TE,39.537387,Petropoulos2016,5.0,B,K07,1,0,0,...,0,1,0,1,0,0,1,0,1,0
E5.1.32,E5.1,early_TE,39.483993,Petropoulos2016,5.0,B,K07,1,1,0,...,0,1,0,1,0,0,1,0,1,0
E5.10.933,E5.10,early_TE,31.499036,Petropoulos2016,5.0,B,K06,1,0,0,...,1,1,0,1,0,0,1,0,1,0
E5.10.934,E5.10,early_TE,39.549130,Petropoulos2016,5.0,B,K06,1,0,0,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E7.9.565,E7.9,medium_TE,48.315030,Petropoulos2016,7.0,B,K08,1,1,0,...,0,1,0,1,0,0,0,1,1,0
E7.9.567,E7.9,medium_TE,45.798088,Petropoulos2016,7.0,B,K08,1,1,0,...,0,1,0,1,0,0,1,0,1,0
E7.9.568,E7.9,late_TE,52.046244,Petropoulos2016,7.0,B,K08,1,1,0,...,0,1,0,1,0,0,1,0,1,0


In [107]:
PKN_early_TE_bin_mtx_with_annotations = PKN_TE_bin_mtx_with_annotations[PKN_TE_bin_mtx_with_annotations['clusterUmap']=='early_TE']
PKN_early_TE_bin_mtx_with_annotations

,Embryo,clusterUmap,Pseudotime,Dataset,EmbryoDay,Stage,clusterEmbryoCells,CDX2,LGALS1,CD3D,...,CNR2,E2F4,CGB3,YY1,EDN1,JAK3,STAT3,CGB8,RBPJ,SOX30
Name,,,,,,,,,,,,,,,,,,,,,
E5.1.28,E5.1,early_TE,37.294838,Petropoulos2016,5.0,B,K07,1,1,0,...,0,1,0,1,0,0,0,0,1,0
E5.1.30,E5.1,early_TE,39.537387,Petropoulos2016,5.0,B,K07,1,0,0,...,0,1,0,1,0,0,1,0,1,0
E5.1.32,E5.1,early_TE,39.483993,Petropoulos2016,5.0,B,K07,1,1,0,...,0,1,0,1,0,0,1,0,1,0
E5.10.933,E5.10,early_TE,31.499036,Petropoulos2016,5.0,B,K06,1,0,0,...,1,1,0,1,0,0,1,0,1,0
E5.10.934,E5.10,early_TE,39.549130,Petropoulos2016,5.0,B,K06,1,0,0,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E6.6.710,E6.6,early_TE,33.099681,Petropoulos2016,6.0,B,K06,1,1,0,...,1,1,0,1,0,0,1,0,1,0
E6.6.713,E6.6,early_TE,35.134309,Petropoulos2016,6.0,B,K06,1,1,0,...,1,1,0,1,0,0,1,0,1,0
E6.6.726,E6.6,early_TE,39.526546,Petropoulos2016,6.0,B,K06,1,1,0,...,0,0,0,1,0,0,0,0,1,0


In [108]:
PKN_medium_TE_bin_mtx_with_annotations = PKN_TE_bin_mtx_with_annotations[PKN_TE_bin_mtx_with_annotations['clusterUmap']=='medium_TE']
PKN_medium_TE_bin_mtx_with_annotations

,Embryo,clusterUmap,Pseudotime,Dataset,EmbryoDay,Stage,clusterEmbryoCells,CDX2,LGALS1,CD3D,...,CNR2,E2F4,CGB3,YY1,EDN1,JAK3,STAT3,CGB8,RBPJ,SOX30
Name,,,,,,,,,,,,,,,,,,,,,
E5.12.1033,E5.12,medium_TE,40.839039,Petropoulos2016,5.0,B,K07,1,0,0,...,0,1,0,1,0,0,1,0,1,0
E5.12.1042,E5.12,medium_TE,43.745943,Petropoulos2016,5.0,B,K07,1,1,0,...,0,1,0,1,0,0,0,0,1,0
E5.13.1767,E5.13,medium_TE,40.829472,Petropoulos2016,5.0,B,K06,1,0,0,...,0,1,0,1,0,0,1,0,1,0
E5.16.1889,E5.16,medium_TE,42.228090,Petropoulos2016,5.0,B,K07,1,1,0,...,0,1,0,1,0,0,0,0,1,0
E5.3.55,E5.3,medium_TE,45.381161,Petropoulos2016,5.0,B,K06,1,1,0,...,0,1,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E7.9.564,E7.9,medium_TE,49.450428,Petropoulos2016,7.0,B,K08,1,1,0,...,0,1,1,1,0,0,1,0,1,0
E7.9.565,E7.9,medium_TE,48.315030,Petropoulos2016,7.0,B,K08,1,1,0,...,0,1,0,1,0,0,0,1,1,0
E7.9.567,E7.9,medium_TE,45.798088,Petropoulos2016,7.0,B,K08,1,1,0,...,0,1,0,1,0,0,1,0,1,0


In [109]:
PKN_late_TE_bin_mtx_with_annotations = PKN_TE_bin_mtx_with_annotations[PKN_TE_bin_mtx_with_annotations['clusterUmap']=='late_TE']
PKN_late_TE_bin_mtx_with_annotations

,Embryo,clusterUmap,Pseudotime,Dataset,EmbryoDay,Stage,clusterEmbryoCells,CDX2,LGALS1,CD3D,...,CNR2,E2F4,CGB3,YY1,EDN1,JAK3,STAT3,CGB8,RBPJ,SOX30
Name,,,,,,,,,,,,,,,,,,,,,
E6.11.1070,E6.11,late_TE,53.427347,Petropoulos2016,6.0,B,K08,0,1,0,...,0,1,0,1,0,0,1,0,1,0
E6.12.1271,E6.12,late_TE,52.980490,Petropoulos2016,6.0,B,K08,1,1,0,...,0,1,0,1,0,0,1,0,1,0
E6.12.1277,E6.12,late_TE,52.978182,Petropoulos2016,6.0,B,K08,1,1,0,...,0,1,1,1,0,0,1,1,1,0
E6.12.1278,E6.12,late_TE,50.623731,Petropoulos2016,6.0,B,K08,1,1,0,...,0,1,0,1,0,0,1,0,1,0
E6.12.1280,E6.12,late_TE,53.560041,Petropoulos2016,6.0,B,K08,1,1,0,...,0,1,1,1,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E7.9.557,E7.9,late_TE,51.045202,Petropoulos2016,7.0,B,K08,1,1,0,...,0,1,1,1,0,0,1,1,1,0
E7.9.559,E7.9,late_TE,50.076372,Petropoulos2016,7.0,B,K08,1,1,0,...,0,1,1,1,0,0,1,0,1,0
E7.9.560,E7.9,late_TE,49.360076,Petropoulos2016,7.0,B,K08,1,1,0,...,0,1,0,1,1,0,1,0,1,0


In [110]:

reduced_early_TE = PKN_early_TE_bin_mtx_with_annotations.iloc[:12,:37]
reduced_medium_TE = PKN_medium_TE_bin_mtx_with_annotations.iloc[:12,:37]
# reduced_medium_TE

B = pd.concat([reduced_early_TE, reduced_medium_TE])
B.to_csv('../matrices/toy_datasets/B.csv', index=True)
B

,Embryo,clusterUmap,Pseudotime,Dataset,EmbryoDay,Stage,clusterEmbryoCells,CDX2,LGALS1,CD3D,...,FOXA1,ZEB1,IL23R,NR2F2,ERCC4,SOD2,CLEC4E,PRDM14,FGFR4,CD3E
Name,,,,,,,,,,,,,,,,,,,,,
E5.1.28,E5.1,early_TE,37.294838,Petropoulos2016,5.0,B,K07,1,1,0,...,0,0,0,0,1,1,0,0,1,0
E5.1.30,E5.1,early_TE,39.537387,Petropoulos2016,5.0,B,K07,1,0,0,...,0,0,0,0,1,1,0,0,1,0
E5.1.32,E5.1,early_TE,39.483993,Petropoulos2016,5.0,B,K07,1,1,0,...,0,0,0,1,0,1,0,0,1,0
E5.10.933,E5.10,early_TE,31.499036,Petropoulos2016,5.0,B,K06,1,0,0,...,0,0,0,0,1,1,0,1,1,0
E5.10.934,E5.10,early_TE,39.549130,Petropoulos2016,5.0,B,K06,1,0,0,...,0,0,0,0,0,0,0,0,1,0
E5.10.935,E5.10,early_TE,39.069345,Petropoulos2016,5.0,B,K06,1,0,0,...,0,0,0,0,0,1,0,1,1,0
E5.10.936,E5.10,early_TE,32.712420,Petropoulos2016,5.0,B,K06,1,1,0,...,0,0,0,0,1,1,0,0,1,0
E5.10.943,E5.10,early_TE,39.061417,Petropoulos2016,5.0,B,K06,1,1,0,...,0,0,0,1,1,1,0,0,1,0
E5.10.944,E5.10,early_TE,36.403583,Petropoulos2016,5.0,B,K06,1,0,0,...,0,0,0,1,0,1,0,1,1,0


In [111]:

reduced_early_TE = PKN_early_TE_bin_mtx_with_annotations.iloc[:25,:107]
reduced_medium_TE = PKN_medium_TE_bin_mtx_with_annotations.iloc[:25,:107]
# reduced_medium_TE

C = pd.concat([reduced_early_TE, reduced_medium_TE])
C.to_csv('../matrices/toy_datasets/C.csv', index=True)
C

,Embryo,clusterUmap,Pseudotime,Dataset,EmbryoDay,Stage,clusterEmbryoCells,CDX2,LGALS1,CD3D,...,RB1CC1,TCF4,CCND1,FGF16,TNIK,SOX17,AKT1,TFCP2L1,ELK1,JDP2
Name,,,,,,,,,,,,,,,,,,,,,
E5.1.28,E5.1,early_TE,37.294838,Petropoulos2016,5.0,B,K07,1,1,0,...,1,0,0,0,1,0,1,1,1,1
E5.1.30,E5.1,early_TE,39.537387,Petropoulos2016,5.0,B,K07,1,0,0,...,1,1,0,0,1,0,1,1,1,1
E5.1.32,E5.1,early_TE,39.483993,Petropoulos2016,5.0,B,K07,1,1,0,...,1,0,0,0,0,0,0,1,1,1
E5.10.933,E5.10,early_TE,31.499036,Petropoulos2016,5.0,B,K06,1,0,0,...,1,1,1,0,0,0,1,1,1,1
E5.10.934,E5.10,early_TE,39.549130,Petropoulos2016,5.0,B,K06,1,0,0,...,1,1,0,0,1,0,0,1,0,1
E5.10.935,E5.10,early_TE,39.069345,Petropoulos2016,5.0,B,K06,1,0,0,...,1,0,0,0,0,0,1,1,1,1
E5.10.936,E5.10,early_TE,32.712420,Petropoulos2016,5.0,B,K06,1,1,0,...,1,0,0,0,1,0,1,1,1,1
E5.10.943,E5.10,early_TE,39.061417,Petropoulos2016,5.0,B,K06,1,1,0,...,1,1,1,0,1,0,1,1,1,1
E5.10.944,E5.10,early_TE,36.403583,Petropoulos2016,5.0,B,K06,1,0,0,...,1,0,1,0,0,1,1,1,1,1


In [112]:

reduced_early_TE = PKN_early_TE_bin_mtx_with_annotations.iloc[:100,:127]
reduced_medium_TE = PKN_medium_TE_bin_mtx_with_annotations.iloc[:100,:127]
# reduced_medium_TE

D = pd.concat([reduced_early_TE, reduced_medium_TE])
D.to_csv('../matrices/toy_datasets/D.csv', index=True)
D

,Embryo,clusterUmap,Pseudotime,Dataset,EmbryoDay,Stage,clusterEmbryoCells,CDX2,LGALS1,CD3D,...,MTSS1,NANOG,BRAF,PVT1,IKBKB,ARNT,IL4,YAP1,VDR,KRAS
Name,,,,,,,,,,,,,,,,,,,,,
E5.1.28,E5.1,early_TE,37.294838,Petropoulos2016,5.0,B,K07,1,1,0,...,0,0,1,1,0,1,0,0,0,1
E5.1.30,E5.1,early_TE,39.537387,Petropoulos2016,5.0,B,K07,1,0,0,...,1,0,1,0,0,1,0,1,0,1
E5.1.32,E5.1,early_TE,39.483993,Petropoulos2016,5.0,B,K07,1,1,0,...,0,0,0,0,1,1,0,1,0,1
E5.10.933,E5.10,early_TE,31.499036,Petropoulos2016,5.0,B,K06,1,0,0,...,0,0,1,1,1,1,0,1,0,1
E5.10.934,E5.10,early_TE,39.549130,Petropoulos2016,5.0,B,K06,1,0,0,...,0,0,0,0,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E6.15.1481,E6.15,medium_TE,46.516472,Petropoulos2016,6.0,B,K08,1,1,0,...,0,0,1,1,1,1,0,1,0,1
E6.17.1570,E6.17,medium_TE,45.744124,Petropoulos2016,6.0,B,K08,1,1,0,...,0,1,0,1,0,1,0,1,0,1
E6.17.1571,E6.17,medium_TE,44.560560,Petropoulos2016,6.0,B,K08,0,1,0,...,0,0,1,1,0,1,0,1,0,1


In [113]:

reduced_early_TE = PKN_early_TE_bin_mtx_with_annotations
reduced_medium_TE = PKN_medium_TE_bin_mtx_with_annotations
# reduced_medium_TE

real = pd.concat([reduced_early_TE, reduced_medium_TE])
real.to_csv('../matrices/real_datasets/real.csv', index=True)
real

,Embryo,clusterUmap,Pseudotime,Dataset,EmbryoDay,Stage,clusterEmbryoCells,CDX2,LGALS1,CD3D,...,CNR2,E2F4,CGB3,YY1,EDN1,JAK3,STAT3,CGB8,RBPJ,SOX30
Name,,,,,,,,,,,,,,,,,,,,,
E5.1.28,E5.1,early_TE,37.294838,Petropoulos2016,5.0,B,K07,1,1,0,...,0,1,0,1,0,0,0,0,1,0
E5.1.30,E5.1,early_TE,39.537387,Petropoulos2016,5.0,B,K07,1,0,0,...,0,1,0,1,0,0,1,0,1,0
E5.1.32,E5.1,early_TE,39.483993,Petropoulos2016,5.0,B,K07,1,1,0,...,0,1,0,1,0,0,1,0,1,0
E5.10.933,E5.10,early_TE,31.499036,Petropoulos2016,5.0,B,K06,1,0,0,...,1,1,0,1,0,0,1,0,1,0
E5.10.934,E5.10,early_TE,39.549130,Petropoulos2016,5.0,B,K06,1,0,0,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E7.9.564,E7.9,medium_TE,49.450428,Petropoulos2016,7.0,B,K08,1,1,0,...,0,1,1,1,0,0,1,0,1,0
E7.9.565,E7.9,medium_TE,48.315030,Petropoulos2016,7.0,B,K08,1,1,0,...,0,1,0,1,0,0,0,1,1,0
E7.9.567,E7.9,medium_TE,45.798088,Petropoulos2016,7.0,B,K08,1,1,0,...,0,1,0,1,0,0,1,0,1,0


In [114]:
df = {
    'Name' :    ['cell1', 'cell2', 'cell3', 'cell4', 'cell5', 'cell6', 'cell7', 'cell8', 'cell9', 'cell10'],
    'Class':    ['c1','c1','c1','c1','c1','c2','c2','c2','c2','c2'],
    'gene1':    [0,1,1,1,0,0,1,0,1,1],
    'gene2':    [1,1,0,0,1,0,0,0,0,1],
    'gene3':    [1,0,1,0,0,1,1,0,1,1],
    'gene4':    [0,1,1,0,1,1,0,0,1,0],
    'gene5':    [1,1,0,1,0,0,1,0,0,0],
    'gene6':    [0,0,1,1,0,0,0,1,0,0],
    'gene7':    [1,0,0,1,1,0,1,1,0,1],
    'gene8':    [0,0,1,0,0,1,0,1,1,1],
    'gene9':    [1,1,0,0,1,1,0,1,0,1],
    'gene10':   [0,1,0,0,1,0,1,1,1,1],
}

A = pd.DataFrame(df)
A.to_csv('../matrices/toy_datasets/A.csv', index=True)
A

,Name,Class,gene1,gene2,gene3,gene4,gene5,gene6,gene7,gene8,gene9,gene10
0,cell1,c1,0,1,1,0,1,0,1,0,1,0
1,cell2,c1,1,1,0,1,1,0,0,0,1,1
2,cell3,c1,1,0,1,1,0,1,0,1,0,0
3,cell4,c1,1,0,0,0,1,1,1,0,0,0
4,cell5,c1,0,1,0,1,0,0,1,0,1,1
5,cell6,c2,0,0,1,1,0,0,0,1,1,0
6,cell7,c2,1,0,1,0,1,0,1,0,0,1
7,cell8,c2,0,0,0,0,0,1,1,1,1,1
8,cell9,c2,1,0,1,1,0,0,0,1,0,1
9,cell10,c2,1,1,1,0,0,0,1,1,1,1
